In [11]:
from collections import Counter
import pandas as pd
import requests
from lxml import etree

In [12]:
# 获取主页json并读取
url = "https://pvp.qq.com/zlkdatasys/data_zlk_xpflby.json"
response_original = requests.get(url).json()

In [13]:
# 拿数据 url_list用于获取皮肤名和英雄名
url_list = []#urllist用来获取英雄名和皮肤名
skin_hero_name_list = []#存储英雄名和皮肤名
skin_release_date_list = []#存储皮肤发布日期

for i in response_original['pcblzlby_c6']:
    url = i['pcblzlbyxqydz_c4']#获取皮肤详情页url
    image_url = "https:" + url#加https开头方便后面读取
    url_list.append(image_url)
    skin_release_date = i['lbyrq_e5']#获取皮肤发布日期
    skin_release_date_list.append(skin_release_date)#存储皮肤发布日期

for j in range(len(url_list)):
    response = requests.get(url_list[j])#获取皮肤详情页
    response.encoding = 'gbk'#设置编码格式
    text = response.text#获取详情页文本
    selector = etree.HTML(text)#解析详情页文本
    skin_hero_name = selector.xpath('//*[@id="showSkin"]/div/div/span/text()')#获取英雄名皮肤名
    skin_hero_name_list.append(skin_hero_name)#存储

# 将皮肤名和英雄名存入并输出为excel文件
for i in range(len(skin_hero_name_list)):
    skin_hero_name_list[i].append(str(skin_release_date_list[i]))

df = pd.DataFrame(skin_hero_name_list,columns=['皮肤名','英雄名','发布日期'])
df.to_excel('王者荣耀皮肤数据.xlsx',sheet_name='Sheet1')#

In [14]:
# 分析每月发布的皮肤数量（为绘图库提供数据）
# 发现一个英雄名是空值（相关网站未写明此英雄名）以及四处数据不一致的情况，手动处理后进行分析
all = pd.read_excel('王者荣耀皮肤数据modified.xlsx')  # 读取文件
time = all["发布日期"].tolist()  # 形成列表
clear_time = [str(i)[0:6] for i in time]  # 提取年月
result = Counter(clear_time)  # 统计每月发布的皮肤数量
d = sorted(result.items(), key=lambda x: x[0], reverse=False)  # 按照年月排序
x1 = [i[0] for i in d]  # 年月（x轴）
y1 = [i[1] for i in d]  # 皮肤数量（y轴）

In [15]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.commons.utils import JsCode
# 绘制每月发布的皮肤数量图
js_formatter = """function (params) {
        return params.seriesData[0].data;
    }"""

(
    Line(init_opts=opts.InitOpts(width="1000px", height="800px"))
    .add_xaxis(
        xaxis_data=x1,

    )
    .add_yaxis(
        series_name="每个月发布的皮肤数量",
        is_smooth=True,
        symbol="emptyCircle",
        is_symbol_show=True,
        color="#d14a61",
        y_axis=y1,
        label_opts=opts.LabelOpts(is_show=True),
        linestyle_opts=opts.LineStyleOpts(width=2),
    )
    .set_global_opts(
        legend_opts=opts.LegendOpts(),
        tooltip_opts=opts.TooltipOpts(trigger="none", axis_pointer_type="cross"),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            axistick_opts=opts.AxisTickOpts(is_align_with_label=True),
            axisline_opts=opts.AxisLineOpts(
                is_on_zero=True, linestyle_opts=opts.LineStyleOpts(color="#d14a61")
            ),
            axispointer_opts=opts.AxisPointerOpts(
                is_show=True, label=opts.LabelOpts(formatter=JsCode(js_formatter))
            ),
        ),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            splitline_opts=opts.SplitLineOpts(
                is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=1)
            ),
        ),
    )
    .render_notebook()
)

In [16]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
from pyecharts import options as opts
from pyecharts.charts import Pie
import pandas as pd
a3=pd.read_excel('职位占比数据.xlsx',sheet_name='Sheet1')#读取文档
position_name=a3['position'].value_counts().keys()#获取position列，计数并分组
position_value=[v for k,v in a3['position'].value_counts().items()]#统计具体的个数


c = (
    Pie()
    .add(
        "",
        [list(z) for z in zip(["战士", "法师", "辅助","刺客","射手","坦克"], position_value)],
        radius=["40%", "55%"],
        label_opts=opts.LabelOpts(
            position="outside",
            formatter="{a|{a}}{abg|}\n{hr|}\n {b|{b}: }{c}  {per|{d}%}  ",
            background_color="#eee",
            border_color="#aaa",
            border_width=1,
            border_radius=4,
            rich={
                "a": {"color": "#999", "lineHeight": 22, "align": "center"},
                "abg": {
                    "backgroundColor": "#e3e3e3",
                    "width": "100%",
                    "align": "right",
                    "height": 22,
                    "borderRadius": [4, 4, 0, 0],
                },
                "hr": {
                    "borderColor": "#aaa",
                    "width": "100%",
                    "borderWidth": 0.5,
                    "height": 0,
                },
                "b": {"fontSize": 16, "lineHeight": 33},
                "per": {
                    "color": "#eee",
                    "backgroundColor": "#334455",
                    "padding": [2, 4],
                    "borderRadius": 2,
                },
            },
        ),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="王者荣耀每个职业的比例"))
)
c.load_javascript()

In [17]:
c.render_notebook()